### Coba Model Bentuk Lain

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import metrics

df=pd.read_csv('pre_title.csv')
df.head(10)

,title,url_berita,sentimen
0,strategi sleman dorong ekonomi pakai batik int...,https://20.detik.com/blak-blakan/20231204-2312...,1
1,greysia polii ajak anak muda terus melangkah r...,https://20.detik.com/demi-indonesia/20231027-2...,1
2,pan x jakcloth pesta anak nongkrong sambangi b...,https://20.detik.com/advertorial/20231109-2311...,1
3,arahan megawati rakernas iv pdip turun rakyatm...,https://20.detik.com/detikupdate/20231001-2310...,1
4,erick thohir ajak muda tak mager indonesia leb...,https://20.detik.com/demi-indonesia/20231027-2...,1
5,pdip sebut ganjar muliakan petani nelayan jadi...,https://20.detik.com/detikupdate/20231001-2310...,1
6,bertemu habib rizieq cak imin bantah bahas duk...,https://20.detik.com/detikupdate/20231001-2310...,1
7,kades bandung mundur dukung amin anies salut,https://20.detik.com/detikupdate/20231001-2310...,1
8,gestur prabowo ditanya soal kans berduet ganjar,https://20.detik.com/detikupdate/20231001-2310...,1
9,anies bertemu alumni itb bahas metode ilmiah m...,https://20.detik.com/detikupdate/20231001-2310...,1


In [14]:
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'
df['sentimen'] = df['sentimen'].map(map_sentiment)


In [15]:
x_train = df['title']
y_train = df['sentimen']

In [16]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(x_train,y_train,test_data_proportion=0.3)

In [17]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,2)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [18]:
#mengunakan model TF-IDF untuk mengekstraksi feature
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(test_corpus)

### Dengan SGDC

In [19]:
from sklearn.linear_model import SGDClassifier

In [20]:
clf = SGDClassifier(loss='hinge', max_iter=100)

In [21]:
clf.fit(tfidf_train_features, train_labels)

SGDClassifier(max_iter=100)

In [22]:
# Predict
clf_pred = clf.predict(tfidf_test_features)

# Accuracy
clf_accuracy = accuracy_score(test_labels, clf_pred)
print("Accuracy:", clf_accuracy)

Accuracy: 0.7080808080808081


In [23]:
print(metrics.classification_report(test_labels, clf_pred))

              precision    recall  f1-score   support

     negatif       0.70      0.70      0.70      1637
      netral       0.72      0.67      0.70      1645
     positif       0.71      0.75      0.73      1668

    accuracy                           0.71      4950
   macro avg       0.71      0.71      0.71      4950
weighted avg       0.71      0.71      0.71      4950



In [27]:
test = ('PSI Akan Bertemu Prabowo, Gerindra: Mudah-mudahan Memberikan Dukungan')
line = tfidf_vectorizer.transform([test])
line.shape
test_result = clf.predict(line)
print("Hasil Prediksi:", test_result) 


Hasil Prediksi: ['positif']
